In [1]:
pip install streamlit_option_menu

In [2]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [3]:
import streamlit as st
import io
from io import BytesIO
import psycopg2
import pandas as pd
import re
from PIL import Image
import numpy as np
from streamlit_option_menu import option_menu
import pytesseract
import base64

In [4]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.


In [5]:
import easyocr

In [6]:
def w():
    q = easyocr.Reader(["en"])
    return q

In [16]:

reader = w()
kizor = Image.open("C:\\Users\\KizorSelvaraj\\Desktop\\project_3\\4.png")
add1= reader.readtext(np.array(kizor),detail=0)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [17]:
add1

['REVANTH',
 'Marketing Executive',
 '123 ABC St,, HYDRABAD, TamilNadu;',
 '600001',
 '+91-456-1234',
 'hello@CHRISTMAS.com',
 'Family',
 'wwW.CHRISTMAS.com',
 'Restaurant']

In [9]:

mydb=psycopg2.connect(host='localhost',user='postgres',password='Kizor@1996',database='BusinessCard_extraction',port=5432)
cursor=mydb.cursor()

create='''create table if not exists bs(id SERIAL PRIMARY KEY,
                                        name varchar(100),
                                        designation varchar(100),
                                        company_name varchar(100),
                                        contact varchar(100),
                                        email varchar(100),
                                        website varchar(100),
                                        address text,
                                        pincode varchar(100),
                                        image_byt bytea)'''
cursor.execute(create)
mydb.commit()

In [18]:
#Extraction

def uploaded_image(Card_img):
    imdict = {'Name': [], 'Designation': [], 'Company': [], 'Contact': [],
                  'Email': [], 'Website': [],
                  'Address': [], 'Pincode': []
                  }
    imdict['Name'].append(add1[0])
    imdict['Designation'].append(add1[1])

    for i in range(2, len(add1)):
        if add1[i].startswith('+') or (add1[i].replace('-', '').isdigit() and '-' in add1[i]):
            imdict['Contact'].append(add1[i])

        elif '@' in add1[i] and '.com' in add1[i]:
            smaller = add1[i].lower()
            imdict['Email'].append(smaller)

        elif 'www' in add1[i] or 'WWW ' in add1[i] or 'wwW' in add1[i]:
            smaller = add1[i].lower()
            imdict['Website'].append(smaller)

        elif 'Tamil Nadu' in add1[i] or 'TamilNadu' in add1[i] or add1[i].isdigit():
            imdict['Address'].append(add1[i])

        elif re.match(r'^[A-Za-z]', add1[i]):
            imdict['Company'].append(add1[i])

        

        else:
            remove_colon = re.sub(r'[.,;]', '', add1[i])
            imdict['Company'].append(remove_colon)

    for key, values in imdict.items():
        if len(values) > 0:
            concat_string = ' '.join(values)
            imdict[key] = [concat_string]
        else:
            values = 'NA'
            imdict[key] = [values]
    return imdict

im=uploaded_image(add1)
image=pd.DataFrame(im)

In [19]:
image

,Name,Designation,Company,Contact,Email,Website,Address,Pincode
0,REVANTH,Marketing Executive,Family Restaurant,+91-456-1234,hello@christmas.com,www.christmas.com,"123 ABC St,, HYDRABAD, TamilNadu; 600001",NA


In [20]:
def image_to_text(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    details =[]
    for i in range(len(result)):
        details.append(result[i][1])
    name = []
    designation = []
    contact =[]
    email =[]
    website = []
    street =[]
    city =[]
    state =[]
    pincode=[]
    company =[]
    
    for i in range(len(details)):
        match1 = re.findall('([0-9]+ [A-Z]+ [A-Za-z]+)., ([a-zA-Z]+). ([a-zA-Z]+)',details[i])    
        match2 = re.findall('([0-9]+ [A-Z]+ [A-Za-z]+)., ([a-zA-Z]+)', details[i])
        match3 = re.findall('^[E].+[a-z]',details[i])
        match4 = re.findall('([A-Za-z]+) ([0-9]+)',details[i])
        match5 = re.findall('([0-9]+ [a-zA-z]+)',details[i])    
        match6 = re.findall('.com$' , details[i])
        match7 = re.findall('([0-9]+)',details[i])
        if details[i] == details[0]:
            name.append(details[i])        
        elif details[i] == details[1]:
            designation.append(details[i])
        elif '-' in details[i]:
            contact.append(details[i])
        elif '@' in details[i]:
            email.append(details[i])
        elif "www " in details[i].lower() or "www." in details[i].lower():
            website.append(details[i])
        elif "WWW" in details[i]:
            website.append(details[i] +"." + details[i+1])
        elif match6:
            pass
        elif match1:
            street.append(match1[0][0])
            city.append(match1[0][1])
            state.append(match1[0][2])
        elif match2:
            street.append(match2[0][0])
            city.append(match2[0][1])
        elif match3:
            city.append(match3[0])
        elif match4:
            state.append(match4[0][0])
            pincode.append(match4[0][1])
        elif match5:
            street.append(match5[0]+' St,')
        elif match7:
            pincode.append(match7[0])
        else:
            company.append(details[i])
    if len(company)>1:
        comp = company[0]+' '+company[1]
        print(comp)
    else:
        comp = company[0]
    if len(contact) >1:
        contact_number = contact[0]
        alternative_number = contact[1]
    else:
        contact_number = contact[0]
        alternative_number = None
    
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
    
    # Convert the binary image data to a base64 encoded string
    encoded_image = base64.b64encode(image_data).decode('utf-8')
    
    image_details = {'name':name[0],'designation':designation[0],'company_name':comp,
                     'contact':contact_number,'alternative':alternative_number,'email':email[0],
                     'website':website[0],'street':street[0],'city':city[0],'state':state[0],
                     'pincode':pincode[0], 'image': encoded_image}
        
    return image_details